# DQN in pytorch

The goal of this exercise is to implement DQN using pytorch and to apply it to the cartpole balancing problem. 

The code is adapted from the Pytorch tutorial: <https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html>.

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install -U gymnasium pygame swig
    !pip install -U moviepy==1.0.3

In [ ]:
# Default libraries
import math
import random
import os
import time
import numpy as np
rng = np.random.default_rng()
import matplotlib.pyplot as plt
from collections import namedtuple, deque

# Gymnasium
import gymnasium as gym
print("gym version:", gym.__version__)

# pytorch
import torch
import torch.nn.functional as F

# Select hardware: 
if torch.cuda.is_available(): # GPU
    device = torch.device("cuda")
elif torch.backends.mps.is_available(): # Metal (Macos)
    device = torch.device("mps")
else: # CPU
    device = torch.device("cpu")
print(f"Device: {device}")

In [ ]:
from moviepy.editor import ImageSequenceClip, ipython_display

class GymRecorder(object):
    """
    Simple wrapper over moviepy to generate a .gif with the frames of a gym environment.
    
    The environment must have the render_mode `rgb_array_list`.
    """
    def __init__(self, env):
        self.env = env
        self._frames = []

    def record(self, frames):
        "To be called at the end of an episode."
        for frame in frames:
            self._frames.append(np.array(frame))

    def make_video(self, filename):
        "Generates the gif video."
        directory = os.path.dirname(os.path.abspath(filename))
        if not os.path.exists(directory):
            os.mkdir(directory)
        self.clip = ImageSequenceClip(list(self._frames), fps=self.env.metadata["render_fps"])
        self.clip.write_gif(filename, fps=self.env.metadata["render_fps"], loop=0)
        del self._frames
        self._frames = []

## Cartpole balancing task

We are going to use the Cartpole balancing problem, which can be loaded with:

```python
gym.make('CartPole-v0', render_mode="rgb_array_list")
```

States have 4 continuous values (position and speed of the cart, angle and speed of the pole) and 2 discrete outputs (going left or right). The reward is +1 for each transition where the pole is still standing (angle of less than 30° with the vertical). 

In CartPole-v0, the episode ends when the pole fails or after 200 steps. In CartPole-v1, the maximum episode length is 500 steps, which is too long for us, so we stick to v0 here.

The maximal (undiscounted) return is therefore 200. Can DQN learn this?

In [ ]:
# Create the environment
env = gym.make('CartPole-v0', render_mode="rgb_array_list")
recorder = GymRecorder(env)

# Sample the initial state
state, info = env.reset()

# One episode:
done = False
return_episode = 0
while not done:

    # Select an action randomly
    action = env.action_space.sample()
    
    # Sample a single transition
    next_state, reward, terminal, truncated, info = env.step(action)

    # End of the episode
    done = terminal or truncated

    # Update undiscounted return
    return_episode += reward
    
    # Go in the next state
    state = next_state

print("Return:", return_episode)
recorder.record(env.render())

In [ ]:
video = "videos/cartpole_random.gif"
recorder.make_video(video)
ipython_display(video)

## Value network in pytorch

As the state in Cartpole has only four dimensions, we do not need a CNN for the value network. A simple MLP with a couple of hidden layers will be enough.

**Q:** Create a MLP class in pytorch taking four inputs and two outputs (one Q-value per action), and two hidden layers of 128 neurons (you can change it later). If possible, make it parameterizable, i.e. have the constructor take in the number of inputs, outputs and hidden neurons. The activation function for the hidden layers should be ReLU.

**Q:** Create a network, an environment, get the initial state using `env.reset()` and pass it to the `forward()` method of your NN. What happens?

Do not forget to send the network to your device, especially if you have a GPU. Create the network using something like:

```python
net = MLP(...).to(device)
```

Alright, we need to cast the state vector into a pytorch tensor, pytorch does not do it automatically.

To cast a numpy vector of shape (4,) into a tensor, one simply needs to call:

```python
state = torch.tensor(state, dtype=torch.float32, device=device)
```

The dtype must be set to `torch.float32` for floating numbers. Integers should be set to `torch.long`. Do not forget to send the tensor to your device if you plan to pass it to your network.

**Q:** Pass the new tensor to your network. What is the shape of the output tensor?

The value network outputs one Q-value per action, great. Now, let's identify the **greedy** action, i.e. the one with the highest Q-value. The two actions expected by the cartpole environment are 0 and 1, i.e. the index of the element with the highest Q-value as a Python integer. 

Have a look at those two methods of `Tensor`:

* ``Tensor.argmax``: <https://pytorch.org/docs/stable/generated/torch.argmax.html>
* ``Tensor.item``: <https://pytorch.org/docs/stable/generated/torch.Tensor.item.html>

**Q:** Find a way to obtain the index (as a Python integer) of the element with the highest value in the tensor of Q-values. Check that it works. 

**Q:** Create a dummy agent class (as in the previous exercises) storing a value network and acting using $\epsilon$-greedy action selection. Add a ``test()``method running a few episodes and possibly recording them. 

The constructor should accept several hyperparameters, such as the `config` dictionary in the following skeleton:

```python
class RandomDQNAgent:
    def __init__(self, env, config):
    def act(self, state):
    def test(self, nb_episodes, recorder=None):
```

but feel free to pass the hyperparameters one by one.

To prepare ourselves, implement a schedule for `epsilon` in the `act()` method: epsilon should start at a high value of 0.9 and decrease exponentially to 0.05 for each action made. The value of epsilon follows this formula:

$$
    \epsilon = 0.05 + (0.9 - 0.05) * \exp ( - \dfrac{t}{1000})
$$

where t is the number of steps since the start. 0.05, 0.9 and 1000 should be parameters of the class.

## Target network

The original DQN algorithm implies two neural networks:

1. The value network $Q_\theta(s, a)$, learning to predict the Q-values for the current state.
2. The target network $Q_{\theta'}(s, a)$, used to predict the Q-values in the next state.

The target network is a copy of the value network (in terms of structure and parameters), but the update occurs only from time to time.

**Q:** Create two MLPs of the same size and predict the Q-values of a single state. What happens?

Obviously, the two MLPs are initialized using random parameters, so they are different. We need a method to copy the weights of a network into another one. 

It is fortunately very easy to save/load the parameters of a pytorch network:

```python
params = net.state_dict()
net.load_state_dict(params)
```

**Q:** Apply these methods to update the weights of the target network with the value one. Check that they now predict the same thing.

## Experience Replay Memory

The second important component of DQN is the experience replay memory (ERM) or replay buffer. It is a limited size buffer that can store $(s, a, r, s', d)$ transitions, where $d$ is a boolean indicating whether the next state $s'$ is terminal or not (in gymnasium, this is the boolean `done = terminal or truncated`).

Below is a simple implementation of an ERM. The important data structure here is `deque` (double-ended queue) which behaves like a list when `append()` is called, until its capacity is reached (`maxlen`), in which case new elements overwrite older ones. 

`batch = sample(batch_size)` randomly samples a minibatch from the ERM and returns a structure of $(s, a, r, s', d)$ transitions, nicely casted into pytorch tensors. These tensors are accessed with `batch.state`, `batch.action`, etc.

In [ ]:
# Named tuples are fancy dictionaries
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'done'))

class ReplayMemory(object):
    "Simple Experience Replay Memory using uniform sampling."

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def append(self, state, action, reward, next_state, done):
        "Appends a transition (s, a, r, s', done) to the buffer."

        # Get numpy arrays even if it is a torch tensor
        if isinstance(state, (torch.Tensor,)): state = state.numpy(force=True)
        if isinstance(next_state, (torch.Tensor,)): next_state = next_state.numpy(force=True)
        
        # Append to the buffer
        self.memory.append(Transition(state, action, reward, next_state, done))

    def sample(self, batch_size):
        "Returns a minibatch of (s, a, r, s', done)"

        # Sample the batch
        transitions = random.sample(self.memory, batch_size)
        
        # Transpose the batch.
        batch = Transition(*zip(*transitions))
        
        # Cast to tensors
        states = torch.tensor(batch.state, dtype=torch.float32, device=device)
        actions = torch.tensor(batch.action, dtype=torch.long, device=device)
        rewards = torch.tensor(batch.reward, dtype=torch.float32, device=device)
        next_states = torch.tensor(batch.next_state, dtype=torch.float32, device=device)
        dones = torch.tensor(batch.done, dtype=torch.bool, device=device)

        return Transition(states, actions, rewards, next_states, dones)

    def __len__(self):
        return len(self.memory)

**Q:** Modify your random DQN agent so that it stores a replay buffer of capacity 10000 and appends all transitions into it. Do a few episodes, sample a small minibatch and have a look at the data you obtain.

**Q:** Use the value network stored into your agent to predict the Q-values of all actions for the states contained in the minibatch. Do NOT use a for loop. Check the size of the resulting tensor. 

**Q:** The previous tensors returns the value of all actions in those visited states. We now want only the Q-value of action that was taken (whose index is in `batch.action`). The resulting tensor should therefore a vector of length `batch_size`. How do we do that?

*Hint:* it would take months of practice to master all the indexing methods available in pytorch: <https://pytorch.org/docs/stable/torch.html#indexing-slicing-joining>. Meanwhile, numpy-style indexing could be useful. Check what the following statements do:

```python
N = 10
A = torch.randn((N, 2))
B = torch.randint(0, 2, (N,))
C = A[range(N), B]
```

## DQN agent

Good, we should now have all the elementary bricks to create our DQN agent. 

Reminder from the lecture:

---

* Initialize value network $Q_{\theta}$ and target network $Q_{\theta'}$.

* Initialize experience replay memory $\mathcal{D}$ of maximal size $N$.

* for $t \in [0, T_\text{total}]$:

    * Select an action $a_t$ based on $Q_\theta(s_t, a)$ ($\epsilon$-greedy), observe $s_{t+1}$ and $r_{t+1}$.

    * Store $(s_t, a_t, r_{t+1}, s_{t+1})$ in the experience replay memory.

    * Every $T_\text{train}$ steps:

        * Sample a minibatch $\mathcal{D}_s$ randomly from $\mathcal{D}$.

        * For each transition $(s_k, a_k, r_k, s'_k)$ in the minibatch:

            * Compute the target value $t_k = r_k + \gamma \, \max_{a'} Q_{\theta'}(s'_k, a')$ using the target network.

        * Update the value network $Q_{\theta}$ on $\mathcal{D}_s$ to minimize:

    $$\mathcal{L}(\theta) = \mathbb{E}_{\mathcal{D}_s}[(t_k - Q_\theta(s_k, a_k))^2]$$

    * Every $T_\text{target}$ steps:

        * Update target network: $\theta' \leftarrow \theta$.
---

Create a DQN agent class inspired from the notebooks on MC or TD. The constructor should create the value and target networks, and make sure that their parameters are the same. It also creates an empty replay buffer. 

The `act()` method implements $\epsilon$-greedy action selection, with an exponentially decaying schedule for $\epsilon$. The greedy action is read from the value network.

The `train()` and `test()` methods run training and test episodes as usual, with optional rendering. The train method should return (or store in the object) the return of each episode (its length) so we can plot it at the end. 

The main difficulty will be the `update()` method, where learning is supposed to happen. It should sample a minibatch from the replay memory, compute a vector of Bellman targets $r_t + \gamma \, \max_a Q(s_{t+1}, a)$ for each transition in the batch, compute the loss function (mse between these targets and the predicted Q-values), backpropagate the gradients and apply the optimizer (Adam, but feel free to pick your preferred optimizer). Refer to the previous notebook on pytorch if you do not know how to do that.

The main tricky part is when $V(s_{t+1})$ has to be predicted by the **target** network. You do not want the target network to learn from the minibatch, so it should not compute the corresponding gradients to save computational time. You can make sure that the target network is purely in inference mode with the following context:

```python
with torch.no_grad():
    next_Q_values = target_net(batch.next_state)
```

Of course you want the Q-value of the greedy action in the next state, not the vector of all Q-values, so check the doc of `Tensor.max()`. 

Importantly, when the next state $s'$ is terminal (either the agent failed or the 200 steps are over), the Bellman target should be simply $r_t$ instead of $r_t + \gamma \, \max_a Q(s_{t+1}, a)$, as no action will be taken in the next state. This is why we saved the booleans `done` were saved in the replay buffer. As they are boolean, you can use them for indexing:

```python
Q = torch.randn((batch_size,))
Q[batch.dones] = 0.0
```

A minor detail: do not start learning until the replay buffer is full enough, otherwise you will not fill your minibatch. Usually, there is no learning until the buffer contains two or three times the batch size. Use `len(memory)` to know the current number of stored transitions.

Here is a set of suggested hyperparameters to help you start. Of course, it is strongly advised to modify them and observe their influence, but it depends on the remaining time.

* $\gamma = 0.99$.
* MLP with two layers of 128 neurons, Adam optimizer with a fixed learning rate of 0.001.
* Replay buffer of maximum capacity 10000, batch size of 128.
* Target network updated every 120 steps.
* Epsilon-greedy action selection, with the schedule:

$$
    \epsilon = 0.05 + (0.9 - 0.05) * \exp ( - \dfrac{t}{1000})
$$

where $t$ is the total number of steps.


**Q:** Train a DQN on cartpole for 250 episodes. How would you characterize the learning process (speed, stability, etc.). If possible, do several runs. Vary the hyperparameters.